<h1><center>

![](https://i.imgur.com/jrRGMfx.png)

AI PRACTITIONER COURSE 

PRACTICE LAB - SESSION 2

**&copy; 2019 VTCA-COTAI. Internal Use Only.**

</center></h1>

# Excercise 1: Ứng dụng K-Means cho bài toán Phân Nhóm Khách Hàng

## **Giới thiệu bài toán ứng dụng K-Mean trong marketing**

![](http://vukhuc.000webhostapp.com/cotai/pics/kmean_bike_db_diagram.png)

1. Yêu cầu bài toán:
 - Chúng ta đóng vai trò là doanh nghiệp sản xuất xe đạp, sản xuất 97 loại xe với các phân loại khác nhau. Khách hàng của chúng ta là những đại lý bán lẻ.
 - Dựa vào dữ liệu lịch sử, phân tích để hiểu những gì đã xảy ra trong quá khứ.
 - Đưa ra khuyến nghị cho chiến lược kinh doanh sắp tới.
2. Ý nghĩa từng table:
 - Bikes (97,6): thông tin các loại xe đạp
 - Bikeshops (30,6): thông tin các đại lý (khách hàng mua xe đạp)
 - Orders (15644,6): đơn hàng bán cho các đại lý (khách hàng mua xe đạp)
3. Kết quả đạt được sau khi làm bài lab:
 - Hiểu được các sử dụng thư viện pandas để xử lý dữ liệu có cấu trúc
 - Biết sử dụng KMean, tìm k tối ưu và cách visualize dữ liệu bằng seaborn
 - Biết cách phân tích một bài toán thực tế ở mức độ đơn giản

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()

##1. Chuẩn bị dữ liệu

**DATASET** 
https://drive.google.com/drive/folders/1uvi36c1YBXbpEokRMwZzZyaqKasqrCoZ?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/Colab Notebooks/S11/kmeans-data'


Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1uvi36c1YBXbpEokRMwZzZyaqKasqrCoZ/kmeans-data


In [ ]:
df_bikes = pd.read_csv('bikes.csv')
print(df_bikes.shape)
df_bikes.head(3)

(97, 6)


,bike.id,model,category1,category2,frame,price
0,1,Supersix Evo Black Inc.,Road,Elite Road,Carbon,12790
1,2,Supersix Evo Hi-Mod Team,Road,Elite Road,Carbon,10660
2,3,Supersix Evo Hi-Mod Dura Ace 1,Road,Elite Road,Carbon,7990


**Kiểm tra xem các cột ``category1``, ``category2``, ``frame`` chứa các giá trị gì**


In [ ]:
# TODO
df_bikes.category1.value_counts()

Mountain    51
Road        46
Name: category1, dtype: int64

In [ ]:
df_bikes.category2.value_counts()

Elite Road            21
Cross Country Race    19
Endurance Road        16
Trail                 13
Sport                  9
Over Mountain          8
Triathalon             5
Cyclocross             4
Fat Bike               2
Name: category2, dtype: int64

In [ ]:
df_bikes.frame.value_counts()

Carbon      57
Aluminum    40
Name: frame, dtype: int64

In [ ]:
df_bikeshops = pd.read_csv('bikeshops.csv')
print(df_bikeshops.shape)
df_bikeshops.head(3)

(30, 6)


,bikeshop.id,bikeshop.name,bikeshop.city,bikeshop.state,latitude,longitude
0,1,Pittsburgh Mountain Machines,Pittsburgh,PA,40.440625,-79.995886
1,2,Ithaca Mountain Climbers,Ithaca,NY,42.443961,-76.501881
2,3,Columbus Race Equipment,Columbus,OH,39.961176,-82.998794


In [ ]:
df_orders = pd.read_csv('orders.csv',index_col=0)
print(df_orders.shape)
df_orders.head(3)

(15644, 6)


,order.id,order.line,order.date,customer.id,product.id,quantity
1,1,1,1/7/2011,2,48,1
2,1,2,1/7/2011,2,52,1
3,2,1,1/10/2011,10,76,1


**TODOs**
1. Tìm cách dùng hàm ``merge(how='inner')`` của pandas để **join** 3 Dataframe lại với nhau


In [ ]:
results = pd.merge(df_bikes,how = 'inner')
# result.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-24-afeb0539adcf>", line 1, in <module>
    results = results.merge(df_bikes,how = 'inner')
NameError: name 'results' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.6/dist-package

NameError: ignored

In [ ]:
df_merge = df_merge.groupby(['bikeshop.name','model','category1','category2','frame','price'])['quantity'].agg(quantity='sum').reset_index()
print(df_merge.shape)
df_merge.head(3)

##2. Viết các hàm tiện ích  

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

def apply_kmeans(X, k):
  kmeans = KMeans(n_clusters = k).fit(X)
  pred = kmeans.labels_
  return pd.concat([X,pd.DataFrame({'index':X.index,'cluster':pred}).set_index('index')], axis=1)

def plot_silhouette(X, max_k, axs):
  sil = []
  K=range(2, max_k)
  for k in K:
    kmeans = KMeans(n_clusters = k).fit(X)
    labels = kmeans.labels_
    sil.append(silhouette_score(X, labels, metric = 'euclidean'))
  # Plot the silhouette_score
  axs.plot(K, sil)
  axs.set_xlabel('k')
  axs.set_ylabel('silhouette score')
  axs.set_title('Silhouette Method')

def plot_elbow(X,n_MaxK,axs):
  K=range(1,n_MaxK)
  W = []
  for k in K:
    kmeans = KMeans(n_clusters = k).fit(X)
    W.append(kmeans.inertia_)
  # Plot the elbow
  axs.plot(K, W)
  axs.set_xlabel('k')
  axs.set_ylabel('Inertia')
  axs.set_title('Elbow Method')

def silhouette_image(X, min_k=2, max_k=10):
  fig, ax = plt.subplots(max_k-min_k+1, 1, figsize=(8,int(max_k * 4)))
  for i in range(min_k, max_k+1):
    ax[i-min_k].grid(False)
    ax[i-min_k].get_yaxis().set_visible(False)
    km = KMeans(n_clusters=i, random_state=42)
    visualizer = SilhouetteVisualizer(km, colors='yellowbrick', ax=ax[i-min_k])
    visualizer.fit(X)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('Optimal k')
plot_silhouette(df_merge[['price']], 10, ax1)
plot_elbow(df_merge[['price']], 10, ax2)

In [ ]:
silhouette_image(df_merge[['price']],2,5)

##3. Thử phân cụm theo giá sản phẩm

In [ ]:
df=df_merge.copy(True)
df_price_temp = apply_kmeans(df[['price']], 2)
df_price_temp.head()

In [ ]:
# TODO
# 1. Tim cach in ra cac gia tri min, max trong df_price_temp theo tung cluster

In [ ]:
df['price'] = pd.cut(df_price_temp.price,bins=[415,4800,12790])
# df['price'] = pd.qcut(df_price_temp.price,2)
df['price'].astype(str)
df

##4. Transform dữ liệu để KMean

In [ ]:
df_t = pd.pivot_table(df, index=['model','category1','category2','frame','price'], columns=['bikeshop.name'], values=['quantity'])
df_t = df_t.reset_index()
df_t

In [ ]:
df_t.drop(columns=['model','category1','category2','frame','price'],inplace=True)
df_t

## 5. Xử lý missing value và scale dữ liệu

In [ ]:
# TODO
# Tim cach fill cac gia tri NaN bang 0
# Apply StandardScaler


df_k = df_t.T.reset_index().drop(columns=['level_0'], axis=1).set_index('bikeshop.name')

## 7. Tìm số cụm tối ưu và phân cụm


### 7.1 Tìm số cụm tối ưu

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,5))
fig.suptitle('Optimal k')
plot_silhouette(df_k,30,ax1)
plot_elbow(df_k,30,ax2)

### 7.2 Thực hiện phân cụm

In [ ]:
df_r = apply_kmeans(df_k, 3)['cluster'].reset_index()
df_r.sort_values(by='cluster')

### 7.3 Vẽ thử các cửa hàng về vị trí địa lý và giá trị cụm của nó
- Chúng ta phân cụm theo số lượng sản phẩm mà của hàng mua nên phân cụm này về lý thuyết sẽ không liên quan đến vị trí địa lý của cửa hàng

In [ ]:
na_loc = pd.merge(df_bikeshops,df_r, how='right',right_on='index', left_on='bikeshop.name')
sns.scatterplot(data=na_loc,x='latitude',y='longitude',hue='cluster')
# na_loc.columns


## 8. Ghép dữ liệu sau khi phân cụm với dữ liệu ban đầu

In [ ]:
df_c = pd.merge(df,df_r,how='left',left_on='bikeshop.name', right_on='index')
df_c

## 9. Plot dữ liệu sau khi phân cụm để hiểu rõ hơn từng cụm (dán nhãn cho từng cụm)

In [ ]:
sns.catplot(data=df_c,x='category1', y='quantity', col='frame', kind='bar',hue='cluster',ci=None)

In [ ]:
filter = (df_c['category1'] == 'Mountain') & (df_c['frame']=='Aluminum')
df_c[filter].sort_values(by=['quantity','cluster'],ascending=False).head(50)

In [ ]:
g=sns.catplot(data=df_c,x='index',y='quantity', col='cluster', kind='bar',hue='frame',ci=None,sharex=False,margin_titles=True)
temp=[plt.setp(ax.get_xticklabels(), rotation=90) for ax in g.axes.flat]

In [ ]:
g=sns.catplot(data=df_c,x='category2',y='quantity',hue='category1',col='cluster', row='frame',kind='bar',ci=False, sharex=False, margin_titles=True,)
# g=sns.catplot(data=df_c,x='category2',y='quantity', col='frame', kind='bar',hue='cluster',ci=None, height=10)
temp=[plt.setp(ax.get_xticklabels(), rotation=90) for ax in g.axes.flat]
g.fig.tight_layout()


In [ ]:
g = sns.catplot(data=df_c,x='category2',y='quantity', col='category1', row='frame', kind='bar',hue='cluster', sharex=False, margin_titles=True,ci=None)
temp=[plt.setp(ax.get_xticklabels(), rotation=90) for ax in g.axes.flat]
g.fig.tight_layout()


# Excercise 2: Data engineering with Titanic Dataset

**DATASET LINK** https://drive.google.com/drive/folders/1C8kZeA0_kwlQG2Oxd3pPqI4I34Fikogi?usp=sharing


## Machine Learning Workflow

1. Define Problem
1. Specify Inputs & Outputs
1. Exploratory Data Analysis
1. Data Collection
1. Data Preprocessing
1. Data Cleaning
1. Visualization
1. Model Design, Training, and Offline Evaluation
1. Model Deployment, Online Evaluation, and Monitoring
1. Model Maintenance, Diagnosis, and Retraining


## Các đặc trưng của tập dữ liệu
Tàu Titanic bị đắm vào **15 tháng 4 năm 1912**, làm chết **1502 / 2224** hành khách và thủy thủ

Trong tập dữ liệu về tàu Titanic này, chúng ta sẽ có **12 đặc trưng như sau**:

Variable | Name	Description 
--- | --- 
Survived |	Survived (1) or died (0)
Pclass |	Passenger's class
Name	| Passenger's name
Sex	| Passenger's sex
Age	| Passenger's age
SibSp	| Number of siblings/spouses aboard
Parch	| Number of parents/children aboard
Ticket	| Ticket number
Fare	| Fare
Cabin	| Cabin
Embarked	| Port of embarkation

Chi tiết: https://www.kaggle.com/c/titanic/data



In [ ]:
import warnings
import json
import sys
import csv
import os


import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import scipy
import numpy as np
np.random.seed(1612)

In [ ]:
sns.set(style='white', context='notebook', palette='deep')
warnings.filterwarnings('ignore')
mpl.style.use('ggplot')
sns.set_style('white')
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Exploratory Data Analysis(EDA)
*   Data Collection
*   Visualization
*   Data Preprocessing
*   Data Cleaning

<img src="http://s9.picofile.com/file/8338476134/EDA.png">

## Data Collection
Đây là quá trình thu thập bất kì dữ liệu nào đáng chú ý, có tiềm năng giải đáp vấn đầ của chúng ta.

In [ ]:
df_train = pd.read_csv('/content/drive/My Drive/COTAI_Training/12. DataEngineer/titanic/train.csv')
df_train.head()

In [ ]:
df_test= pd.read_csv('/content/drive/My Drive/COTAI_Training/12. DataEngineer/titanic/test.csv')
df_test.head()

><font color="red"><b>Note: </b></font>

* Cách gọi khác của 1 dòng **dòng**: observation, sample, example, instance, record
* Cách gọi khác của 1 **cột**: feature, attribute

Cách xem ngẫu nhiên một vài dòng trong tập dữ liệu

In [ ]:
df_train.sample(5) 

Xem qua thống kê sơ bộ của các cột bằng describe()

In [ ]:
df_train.describe() 


## Visualization


**Lọc ra những giá trị có thể xuất hiện trong từng cột**

In [ ]:
# TODO
# 1. Print out all unique values



**Đếm số lượng unique value**

In [ ]:
# TODO

### Count Plot

**Survived vs Sex**

In [ ]:
plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
sns.countplot(df_train.Sex)
plt.title('Count plot for Sex')

plt.subplot(1,3,2)
sns.countplot(df_train.Survived)
plt.title('Number of passenger Survived')

plt.subplot(1,3,3)
sns.countplot(x="Survived", hue="Sex", data=df_train)
plt.title('Number of passenger Survived')

**Survived vs Pclass**

In [ ]:
# TODO

### Scatter Plot

[Scatter plot](https://en.wikipedia.org/wiki/Scatter_plot) Xác định quan hệ giữa 2 đặc trưng.



**Survived vs Pclass vs Fare vs Age**

In [ ]:
#https://seaborn.pydata.org/generated/seaborn.FacetGrid.html

g = sns.FacetGrid(df_train, hue="Survived", col="Pclass", margin_titles=True,
                  palette={1:"seagreen", 0:"gray"}) 
g=g.map(plt.scatter, "Fare", "Age", edgecolor="w").add_legend();

**Survived vs Pclass vs SibSp vs Parch**

In [ ]:
# TODO


### Box
Box là phương pháp mô ta dữ liệu số thông qua những phần tư của chúng.
Chi tiết: 
- https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/box-whisker-plots/a/box-plot-review
- https://www.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/box-whisker-plots/a/identifying-outliers-iqr-rule
<img src="https://miro.medium.com/max/13500/1*2c21SkzJMf3frPXPAR_gZA.png">


**Example**

![boxplot](https://i.imgur.com/Mcw6vXv.png)

In [ ]:
ax= sns.boxplot(x="Pclass", y="Age", data=df_train)
plt.show()


### Histogram

In [ ]:
# histograms
df_train.hist(figsize=(15,10),bins=40)
plt.figure()

### Correlation Heatmap

In [ ]:
# TODO
# 1. Find correlation matrix using df.corr()
# 2. Plot heatmap


## Data Preprocessing
**Data preprocessing** là kĩ thuật chuẩn hóa, làm sạch dữ liệu trước khi đưa vào thuật toán.
Một vài bước tiêu biểu của Data preprocessing:
* Xử lý dữ liệu không cân bằng
* Xử lý dữ liệu bị thiếu
* Xử lý nhiễu (https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba)
* Chuẩn hóa dữ liệu (https://medium.com/@urvashilluniya/why-data-normalization-is-necessary-for-machine-learning-models-681b65a05029)
* Chọn lọc đặc trưng (https://medium.com/analytics-vidhya/feature-selection-using-scikit-learn-5b4362e0c19b)

 ## Types of features
Features:
* số 
* theo từng mục (category)
* ngày tháng (datetime)

### Hãy xác định dạng đặc trưng trong tập dữ liệu:


In [ ]:
# TODO
# 1. check data type of each columns


## Data exploration

In [ ]:
# shape
print(df_train.shape)

>  <font color="red"><b>Note:</b></font>
how many NA elements in every column


In [ ]:
df_train.isnull()

In [ ]:
# TODO 
# 1. Check null on df_train and df_test

**Xóa cột**

In [ ]:
# TODO
# 1. Drop 4 columns Embarked, PassengerId, Cabin, Ticket


**Xóa những dòng có giá trị NA**

In [ ]:
# remove rows that have NA's
df_test['Fare'] = df_test['Fare'].dropna()

**Replace giá trị NA bằng giá trị khác**

In [ ]:
df_train.loc[df_train.Age.isnull(), 'Age'] = df_train.groupby("Pclass").Age.transform('median')

df_test.loc[df_train.Age.isnull(), 'Age'] = df_test.groupby("Pclass").Age.transform('median')

## Feature transformations
Chúng ta sẽ chuyển đổi dữ liệu sang một dạng mới với kì vọng sẽ đem lại giá trị tốt hơn cho việc phân tích, đưa ra quyết định
Ở đây chúng ta sẽ biến đổi:
1. Name
1. Age
1. SibSp & Parch



**Transform Name**

In [ ]:
df_train['Name'].head(10)

In [ ]:
df_train['Name'] = df_train.Name.str.extract(' ([A-Za-z]+)\.', expand = False)
df_test['Name'] = df_test.Name.str.extract(' ([A-Za-z]+)\.', expand = False)

In [ ]:
df_train['Name'].unique().tolist()

In [ ]:
df_train.rename(columns={'Name' : 'Title'}, inplace=True)
df_train['Title'] = df_train['Title'].replace(['Rev', 'Dr', 'Col', 'Ms', 'Mlle', 'Major', 'Countess', 
                                       'Capt', 'Dona', 'Jonkheer', 'Lady', 'Sir', 'Mme', 'Don'], 'Other')
                                      
df_test.rename(columns={'Name' : 'Title'}, inplace=True)
df_test['Title'] = df_test['Title'].replace(['Rev', 'Dr', 'Col', 'Ms', 'Mlle', 'Major', 'Countess', 
                                       'Capt', 'Dona', 'Jonkheer', 'Lady', 'Sir', 'Mme', 'Don'], 'Other')

In [ ]:
df_train.Title.value_counts()

**Transform Age**

In [ ]:
bins = (0, 5, 12, 18, 25, 35, 60, 120)
group_names = [ 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Senior']
categories = pd.cut(df_train.Age, bins, labels=group_names)
df_train.Age = categories
df_test.Age = categories

In [ ]:
df_train.Age.value_counts()

**Transform SibSp & Parch**

In [ ]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1

In [ ]:
df_train = df_train.drop(columns=['SibSp','Parch'])
df_test = df_test.drop(columns=['SibSp','Parch'])

In [ ]:
df_train.FamilySize

## Feature Encoding

Apply One Hot Encoding for all categorical columns

In [ ]:
df_train.head()

**Encode Sex column**

In [ ]:
df_train['Sex'] = df_train['Sex'].map({'male':1,'female':0})
df_test['Sex'] = df_test['Sex'].map({'male':1,'female':0})

In [ ]:
df_train.head()

**Encode other columns**

In [ ]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)
df_train.head()

## Prepare dataset

**Lấy ra label y**

In [ ]:
# TODO
# 1. get y from column Suvived
# 2. drop column Survived

**Train / Validation Split**

In [ ]:
# TODO
# test_size=0.3 and stratify=y

**Feature Scaling**

In [ ]:
# TODO




## 2D Visualization

### PCA

In [ ]:
# TODO
# 1. import PCA
# 2. Initializa pca with 2 components
# 3. Apply pca on X_train
# 3. Name the new variable df_pca


In [ ]:
plt.figure(figsize =(8, 8))
sns.scatterplot(df_pca[:,0], df_pca[:,1], hue=y_train, legend='full')

### T-SNE

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE()
df_tsne = tsne.fit_transform(X_train)

plt.figure(figsize =(8, 8))
sns.scatterplot(df_tsne[:,0], df_tsne[:,1], hue=y_train, legend='full')

## Ensemble Model

### Logistic Regression

In [ ]:
# TODO
# 1. Define LogisticRegression model
# 2. Fit on Train


In [ ]:
# TODO
# 1. Use score() function on 2 Sets


### Support Vector Machine

In [ ]:
C_values = [0.01, 0.1, 1, 3]
gamma_values = [0.01, 0.1, 1, 3]
kernel_values = ['linear', 'poly', 'rbf']

param_grid = {
    'kernel': kernel_values,
    'C': C_values,
    'gamma': gamma_values
}

In [ ]:
# TODO
# 1. import SVC, GridSearchCV
# 3. khởi tạo GridSearchCV với số k-fold (cv) = 3
# 4. fit trên Train Set 
# 5. lấy ra best model
# 6. 


In [ ]:
# TODO
# 1. Use score() function on 2 Sets


### Decision Tree

In [ ]:
params = {
    'criterion': ['entropy','gini'],
    'max_depth': [3,5,7],
    'min_samples_split': np.linspace(0.1, 1.0, 10), 
    'max_features':  ['auto', 'log2']
}

# TODO
# 1. import Decision Tree
# 3. khởi tạo GridSearchCV với số k-fold (cv) = 2
# 4. fit trên Train Set 
# 5. lấy ra best model



In [ ]:
# TODO
# 1. Use score() function on 2 Sets


### Random Forest

In [ ]:
 param_grid_random={'criterion': ['gini', 'entropy'],
            'max_depth': [3, 5, 7],
            'max_features': ['auto', 'log2'],
            'n_estimators': [100, 300, 500]})

# TODO
# 1. Import RandomForestClassifier
# 2. khởi tạo GridSearchCV với số k-fold (cv) = 2
# 3. fit trên Train Set 
# 4. lấy ra best model



In [ ]:
# TODO
# 1. Use score() function on 2 Sets


### Voting Classifier

voting = {'hard', 'soft'}

If 'hard', uses predicted class labels for majority rule voting. Else if 'soft', predicts the class label based on the argmax of the sums of the predicted probabilities, which is recommended for an ensemble of well-calibrated classifiers.

In [ ]:
from sklearn.ensemble import VotingClassifier

ensemble = VotingClassifier(estimators=[
                              ('Logistic Regression', logreg), 
                              ('SVM', svc), 
                              ('Decision Tree', decision_tree), 
                              ('Random Forest', random_forest)
                              ],
                            voting='hard',
                            n_jobs=-1)



ensemble.fit(X_train, y_train)

In [ ]:
# TODO
# 1. Use score() function on 2 Sets


**Confusion Matrix**

In [ ]:
# TODO
# 1. Import confusion matrix from sklearn
# 2. Use confusion_matrix to draw a heatmap
# 3. if your heatmap show 8e+2 numbers. Insise heatmap() function, put a parameter fmt='.1f'  
